<a href="https://colab.research.google.com/github/sccn/sound2meg/blob/main/Spatial_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import os
from torch.autograd import Variable
import math
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.optim as optim
from scipy.io import loadmat
from dataset_loading import Sound2MEGDataset
from torch.utils.data import Dataset, DataLoader, random_split
import gc

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.__version__)

cuda:0
2.0.0


In [3]:
import sys
sys.tracebacklimit = 0

In [4]:
class SpatialAttention(nn.Module):
  def __init__(self,in_channels, out_channels, K, path):
    super(SpatialAttention, self).__init__()
    self.out = out_channels
    self.input = in_channels
    self.K = K
    self.z = Parameter(torch.randn(self.out, K*K, dtype = torch.cfloat)/(32*32))
    self.z.requires_grad = True
    self.positions = loadmat(path + 'electrode_positions.mat')
    self.positions = self.positions['positions']
    self.x = torch.tensor(self.positions[:, 0]).to(device)
    self.y = torch.tensor(self.positions[:, 1]).to(device)
    self.cos = []
    self.sin = []
    for i in range(in_channels):
      self.cos_v = []
      self.sin_v = []
      for k in range(K):
        for l in range(K):
          self.cos_v.append(torch.cos(2*math.pi*(k*self.x[i]+l*self.y[i])))
          self.sin_v.append(torch.sin(2*math.pi*(k*self.x[i]+l*self.y[i])))
      self.cos.append(torch.stack(self.cos_v))
      self.sin.append(torch.stack(self.sin_v))
    self.cos = torch.stack(self.cos).to(device)
    self.sin = torch.stack(self.sin).to(device)
  def forward(self, X):
    N = X.size()[0]
    SA = torch.zeros(N, 270, 360).to(device)
    z_r = self.z.real
    z_i = self.z.imag
    a = (torch.mm(z_r.float(), torch.transpose(self.cos, 0, 1).float()) + torch.mm(z_i.float(), torch.transpose(self.sin, 0, 1).float())).to(device)
    exp2 = torch.sum(torch.exp(a[:, 0:self.out]), 1).to(device)
    exp2 = torch.transpose(exp2.unsqueeze(0), 0, 1)
    exp2 = torch.mm(exp2, torch.ones(1, 360).to(device))
    for i in range(N):
      exp1 = torch.mm(torch.exp(a), X[i]).to(device)
      SA[i] = (exp1/exp2).to(device)
      #SA[i] = SpatialAttentionSoftmax(self.input, self.out, X[i], a)
    return SA

In [5]:
class SubjectLayer(nn.Module):
  def __init__(self):
    super(SubjectLayer, self).__init__()
    self.layers = []

    for i in range(124): #124 subjects
      layer = nn.Conv2d(270, 270, 1).to(device)
      self.layers.append(layer)
      
  def forward(self, x, s_idx):
    x = x.unsqueeze(1)
    for i in range(len(x)):
      x[i] = self.layers[s_idx[i]](x[i].clone())
    return x[:, 0, :, :]

In [ ]:
class Net(nn.Module):
  def __init__(self, path, F):
    super(Net, self).__init__()
    self.SA = SpatialAttention(273, 270, 32, path)
    self.Subject = SubjectLayer().to(device)
    self.F = F
  def forward(self, x, s_idx):
    x = self.SA(x).unsqueeze(0)
    x = torch.permute(x, (1, 2, 3, 0)) # subject attention?
    x = nn.Conv2d(270, 270, (1, 1)).to(device)(x)
    x = self.Subject(x, s_idx)
    for k in range(1,6):
      p = pow(2,(2*k)%5)
      q = pow(2,(2*k+1)%5)
      if k == 1:
        x = nn.Conv2d(270, 320, (3, 1), dilation = 1, padding = (1, 0)).to(device)(x)
        x = nn.BatchNorm2d(320).to(device)(x)
        x = nn.GELU().to(device)(x)
        x = nn.Conv2d(320, 320, (3, 1), dilation = 1, padding = (1, 0)).to(device)(x)
        x = nn.BatchNorm2d(320).to(device)(x)
        x = nn.GELU().to(device)(x)
        x = nn.Conv2d(320, 640, (3, 1), dilation = 2, padding = (2, 0)).to(device)(x)
        x = torch.transpose(x, 3, 1)
        x = nn.GLU().to(device)(x)
        x = torch.transpose(x, 3, 1)
      else:
        x1 = nn.Conv2d(320, 320, (3, 1), dilation = p, padding = (p, 0)).to(device)(x)
        x1 = nn.BatchNorm2d(320).to(device)(x1)
        x1 = nn.GELU().to(device)(x1)
        x2 = x + x1
        x3 = nn.Conv2d(320, 320, (3, 1), dilation = q, padding = (q, 0)).to(device)(x2)
        x3 = nn.BatchNorm2d(320).to(device)(x3)
        x3 = nn.GELU().to(device)(x3)
        x4 = x2 + x2
        x_out = nn.Conv2d(320, 640, (3, 1), dilation = 2, padding = (2, 0)).to(device)(x4)
        x_out = torch.transpose(x_out, 3, 1)
        x_out = nn.GLU().to(device)(x_out)
        x_out = torch.transpose(x_out, 3, 1)
    x_out = nn.Conv2d(320, 640, (1, 1)).to(device)(x_out)
    x_out = nn.GELU().to(device)(x_out)
    x_out = nn.Conv2d(640, self.F, (1, 1)).to(device)(x_out)
    return x_out

In [6]:
class Net(nn.Module):
  def __init__(self, path, F):
    super(Net, self).__init__()
    self.SA = SpatialAttention(273, 270, 32, path)
    self.Subject = SubjectLayer().to(device)
    self.F = F
    self.conv1 = nn.Conv2d(270, 270, (1, 1)).to(device)
    self.conv2 = nn.Conv2d(320, 640, (1, 1)).to(device)
    self.conv3 = nn.Conv2d(640, self.F, (1, 1)).to(device)
    self.gelu = nn.GELU().to(device)
    self.loop_convs = []
    self.loop_batchnorms = []
    self.loop_gelus = []
    self.loop_glus = []
    for k in range(1, 6):
      p = pow(2,(2*k)%5)
      q = pow(2,(2*k+1)%5)
      self.convs = []
      self.batchnorms = []
      self.gelus = []
      self.convs.append(nn.Conv2d(320, 320, (3, 1), dilation = p, padding = (p, 0)).to(device))
      self.convs.append(nn.Conv2d(320, 320, (3, 1), dilation = q, padding = (q, 0)).to(device))
      self.convs.append(nn.Conv2d(320, 640, (3, 1), dilation = 2, padding = (2, 0)).to(device))
      for i in range(2):
        self.batchnorms.append(nn.BatchNorm2d(320).to(device))
        self.gelus.append(nn.GELU().to(device))
      self.loop_convs.append(self.convs)
      self.loop_batchnorms.append(self.batchnorms)
      self.loop_gelus.append(self.gelus)
      self.loop_glus.append(nn.GLU().to(device))
    self.loop_convs[0][0] = nn.Conv2d(270, 320, (3, 1), dilation = 1, padding = (1, 0)).to(device)
  def forward(self, x, s_idx):
    x = self.SA(x).unsqueeze(3)
    x = self.conv1(x)
    x = self.Subject(x, s_idx)
    for i in range(5):
      if i == 0:
        x = self.loop_convs[0][0](x)
        x = self.loop_batchnorms[0][0](x)
        x = self.loop_gelus[0][0](x)
        x = self.loop_convs[0][1](x)
        x = self.loop_batchnorms[0][1](x)
        x = self.loop_gelus[0][1](x)
        x = self.loop_convs[0][2](x)
        x = torch.transpose(x, 3, 1)
        x = self.loop_glus[0](x)
        x = torch.transpose(x, 3, 1)
      else:
        x1 = self.loop_convs[i][0](x)
        x1 = self.loop_batchnorms[i][0](x)
        x1 = self.loop_gelus[i][0](x)
        x2 = x + x1
        x3 = self.loop_convs[i][1](x2)
        x3 = self.loop_batchnorms[i][1](x3)
        x3 = self.loop_gelus[i][1](x3)
        x4 = x2 + x3
        x5 = self.loop_convs[i][2](x4)
        x5 = torch.transpose(x5, 3, 1)
        x5 = self.loop_glus[i](x5)
        x = torch.transpose(x5, 3, 1)
    x_out = self.conv2(x)
    x_out = self.gelu(x_out)
    x_out = self.conv3(x_out)
    return x_out

In [7]:
def CLIP_loss(Z, Y):
  N = Y.size(dim = 0)
  #inner_product = torch.zeros(N, N)
  log_softmax = torch.zeros(N).to(device)
  Z_row = torch.reshape(Z, (N, -1)).to(device)
  Y_row = torch.reshape(Y, (N, -1)).to(device)
  inner_product = (torch.mm(Z_row, torch.transpose(Y_row, 1, 0))/(N*N)).to(device)
  for i in range(N):
    inn = inner_product[i, :].to(device)
    log_softmax[i] = torch.log(nn.functional.softmax(inn, -1))[i]
  return sum(-1*log_softmax)

In [9]:
Dataset = Sound2MEGDataset('/expanse/projects/nsg/external_users/public/arno/')
training_data, validation_data, test_data = random_split(Dataset, [11497, 3285, 1642], generator=torch.Generator().manual_seed(42))
Training_Data_Batches = DataLoader(training_data, batch_size = 2, shuffle = True)
Validation_Data_Batches = DataLoader(validation_data, batch_size = 128, shuffle = True)
BrainModule = Net('/expanse/projects/nsg/external_users/public/arno/', 120)
BrainModule.to(device)
optimizer = optim.Adam(BrainModule.parameters(), lr = 0.0003)
loss_train = []
loss_val = []

for i in range(1):
  loss_t = 0
  loss_v = 0
  for MEG, WAV, Sub in Training_Data_Batches:
    Sub = Sub.tolist()
    Z = BrainModule(MEG.to(device), Sub)
    Z = Z[:, :, :, 0]
    loss = CLIP_loss(Z.float(), WAV.abs().float().to(device))
    print(loss.item())
    torch.autograd.set_detect_anomaly(True)
    optimizer.zero_grad()
    loss.backward()
    loss_t = loss_t + loss.item()
    optimizer.step()
  print(loss_t/len(Training_Data_Batches))
  loss_train.append(loss_t/len(Training_Data_Batches))
  for MEG_val, WAV_val, Sub_val in Validation_Data_Batches:
    Z_val = BrainModule(MEG_val.to(device), Sub_val)
    loss = CLIP_loss(Z_val.float(), WAV_val.abs().float().to(device))
    print(loss.item())
    loss_v = loss_v + loss.item()
  print(loss_v/len(Validation_Data_Batches))
  loss_val.append(loss_v/len(Validation_Data_Batches))
  gc.collect()
  torch.cuda.empty_cache()

print(loss_train)
print(loss_val)

1.3853585720062256
1.3906989097595215
1.394761323928833
1.3911864757537842
1.3856446743011475
1.3858895301818848
1.388353705406189
1.388444185256958
1.3879313468933105
1.3867535591125488
1.386294960975647
1.3855907917022705
1.383887767791748
1.3774429559707642
1.3848984241485596
1.3931632041931152
1.3835711479187012
1.3870904445648193
1.3851640224456787
1.3838481903076172
1.3854296207427979
1.3791072368621826
1.3843356370925903
1.381448745727539
1.3891133069992065
1.387267827987671
1.3767356872558594
1.3977491855621338
1.3868671655654907
1.388546347618103
1.3883283138275146
1.3810768127441406
1.38742995262146
1.3825386762619019
1.38352632522583
1.3724682331085205
1.3854873180389404
1.3801610469818115
1.3807604312896729
1.3884484767913818
1.3954447507858276
1.394754409790039
1.3794465065002441
1.3801205158233643
1.3859058618545532
1.3932130336761475
1.3746304512023926
1.3851118087768555
1.3870270252227783
1.3877451419830322
1.3898602724075317
1.379781723022461
1.3728654384613037
1.39619

1.3830591440200806
1.3529316186904907
1.363905429840088
1.361361026763916
1.3861682415008545
1.3912391662597656
1.3948605060577393
1.3814928531646729
1.3828668594360352
1.3733365535736084
1.419045090675354
1.3389184474945068
1.3931267261505127
1.418747901916504
1.3704450130462646
1.3939908742904663
1.3805150985717773
1.360273838043213
1.3711628913879395
1.3162648677825928
1.3734831809997559
1.3635497093200684
1.388005018234253
1.357571005821228
1.3735692501068115
1.336181640625
1.4113388061523438
1.394700288772583
1.3777257204055786
1.3852450847625732
1.3175044059753418
1.3234081268310547
1.3981926441192627
1.380265235900879
1.4086790084838867
1.4679231643676758
1.376204490661621
1.384925127029419
1.381052017211914
1.2828478813171387
1.3341500759124756
1.4329432249069214
1.3721373081207275
1.3818284273147583
1.3754732608795166
1.3855031728744507
1.415371298789978
1.3775341510772705
1.3869826793670654
1.3689178228378296
1.4208195209503174
1.451083779335022
1.4150052070617676
1.336037516

1.3747570514678955
1.5314525365829468
1.3824927806854248
1.4197654724121094
1.3832966089248657
1.3981424570083618
1.4063040018081665
1.3768895864486694
1.2954447269439697
1.3749730587005615
1.4445899724960327
1.3271682262420654
1.3434556722640991
1.3972983360290527
1.4526619911193848
1.3955708742141724
1.4460186958312988
1.4369897842407227
1.44240140914917
1.3176050186157227
1.6180784702301025
1.2452207803726196
1.4440021514892578
1.3808536529541016
1.4014211893081665
1.3528257608413696
1.3619980812072754
1.4147233963012695
1.3586649894714355
1.2857086658477783
1.3718422651290894
1.3891656398773193
1.3425252437591553
1.438650369644165
1.383748173713684
1.4334845542907715
1.292015552520752
1.4266659021377563
1.4523056745529175
1.338348627090454
1.4052832126617432
1.4389344453811646
1.341493844985962
1.2972018718719482
1.4149632453918457
1.5728168487548828
1.392240285873413
1.364086389541626
1.3923428058624268
1.307502269744873
1.4563448429107666
1.3892426490783691
1.4156731367111206
1.3

1.3952980041503906
1.3525981903076172
1.3522732257843018
1.4352598190307617
1.4136239290237427
1.3623051643371582
1.3900541067123413
1.4324936866760254
1.3691140413284302
1.380072832107544
1.412170171737671
1.3765345811843872
1.5170692205429077
1.3652055263519287
1.3205174207687378
1.3711819648742676
1.3884081840515137
1.379654884338379
1.3914289474487305
1.4494379758834839
1.361228585243225
1.3844941854476929
1.4142553806304932
1.3548808097839355
1.4306659698486328
1.3514903783798218
1.4384446144104004
1.4333860874176025
1.3820117712020874
1.4319508075714111
1.3791587352752686
1.3862571716308594
1.421684741973877
1.371378779411316
1.3302104473114014
1.4436131715774536
1.4156043529510498
1.385915756225586
1.3590303659439087
1.380446434020996
1.3427953720092773
1.3599531650543213
1.299877643585205
1.3884804248809814
1.3895941972732544
1.3772573471069336
1.4024758338928223
1.3636066913604736
1.3213719129562378
1.3243513107299805
1.3497719764709473
1.2962443828582764
1.5244158506393433
1.

1.38029146194458
1.4028735160827637
1.395867109298706
1.401902675628662
1.4007337093353271
1.3677432537078857
1.3843472003936768
1.4066028594970703
1.366697907447815
1.373847246170044
1.3963744640350342
1.4011645317077637
1.405097246170044
1.3404016494750977
1.3759541511535645
1.3917311429977417
1.3771092891693115
1.4116837978363037
1.4078123569488525
1.343876600265503
1.421532154083252
1.3884167671203613
1.385650634765625
1.3580535650253296
1.3489629030227661
1.3875324726104736
1.3677325248718262
1.4152946472167969
1.3942086696624756
1.3781158924102783
1.3712167739868164
1.4297930002212524
1.4457356929779053
1.3862184286117554
1.3389643430709839
1.3789966106414795
1.3859317302703857
1.3668599128723145
1.3676396608352661
1.3842799663543701
1.3673579692840576
1.4012532234191895
1.4378139972686768
1.3631688356399536
1.349374771118164
1.3908615112304688
1.3909597396850586
1.374709129333496
1.4003537893295288
1.4045664072036743
1.3788807392120361
1.3839623928070068
1.4061282873153687
1.344

...

1.3221721649169922
1.419067621231079
1.3998854160308838
1.4158363342285156
1.3353683948516846
1.430936574935913
1.396335482597351
1.368383765220642
1.3647117614746094
1.5282843112945557
1.342780351638794
1.342350959777832
1.4304957389831543
1.3385721445083618
1.3813471794128418
1.4077152013778687
1.4018173217773438
1.538583755493164
1.4377329349517822
1.3079266548156738
1.2017520666122437
1.2254397869110107
1.4578620195388794
1.592308521270752
1.3898625373840332
1.441831350326538
1.2500133514404297
1.4032838344573975
1.4170136451721191
1.3383586406707764
1.1276822090148926
1.2567391395568848
1.2848166227340698
1.3415552377700806
1.5373435020446777
1.4578111171722412
1.1326251029968262
1.1742048263549805
1.2968239784240723
1.3703687191009521
1.4449810981750488
1.4067027568817139
1.3033301830291748
1.1883244514465332
1.2210134267807007
1.1818675994873047
1.4606784582138062
1.2811440229415894
1.4873545169830322
1.3155922889709473
1.425765037536621
1.2897083759307861
1.3341519832611084
1.3

1.469459056854248
1.407203197479248
1.2508752346038818
1.3490923643112183
1.3911466598510742
1.5144765377044678
1.4738504886627197
1.2496280670166016
1.357796311378479
1.4029371738433838
1.252084732055664
1.5656185150146484
1.4307136535644531
1.3299705982208252
1.312957525253296
1.254616141319275
1.2968974113464355
1.0577009916305542
1.365162968635559
1.2678046226501465
1.4253969192504883
1.5019011497497559
1.3551721572875977
1.2070128917694092
1.255253791809082
1.6041748523712158
1.3610987663269043
1.3642547130584717
1.4441511631011963
1.3912725448608398
1.3797321319580078
1.6767692565917969
1.3291040658950806
1.6663199663162231
1.3089237213134766
1.4207980632781982
1.3672699928283691
1.3484578132629395
1.1574294567108154
1.471576452255249
1.4891042709350586
1.5527780055999756
1.2235890626907349
1.49271559715271
1.3742600679397583
1.3656911849975586
1.2903404235839844
1.4777750968933105
1.4324650764465332
1.4578497409820557
1.3981540203094482
1.3698903322219849
1.3538994789123535
1.39

1.3285791873931885
1.3214937448501587
1.4493881464004517
1.2575864791870117
1.3288437128067017
1.4060068130493164
1.3813540935516357
1.1104012727737427
1.3787482976913452
1.1394078731536865
1.5149128437042236
1.303165316581726
1.3510445356369019
1.2954747676849365
1.0903797149658203
1.502501368522644
1.304050087928772
1.7515754699707031
1.503669023513794
1.2880051136016846
1.4492805004119873
1.7951627969741821
1.3451859951019287
1.1515086889266968
1.389622449874878
1.1723674535751343
1.5652918815612793
1.3040555715560913
1.2995893955230713
1.3657869100570679
1.3005025386810303
1.220857858657837
1.2228829860687256
1.4570900201797485
1.2776607275009155
1.2729562520980835
1.485100507736206
1.4261062145233154
1.4080666303634644
1.3844921588897705
1.2969837188720703
1.2363500595092773
1.0941076278686523
1.4732074737548828
1.3994286060333252
1.4781146049499512
1.3749216794967651
1.3560737371444702
1.4644203186035156
1.222815990447998
1.3463037014007568
1.4218178987503052
1.642331838607788
1.

1.4823436737060547
1.0877623558044434
1.2536523342132568
1.2919557094573975
1.49582839012146
1.2962348461151123
1.4537286758422852
1.2647178173065186
1.0960958003997803
1.447300910949707
1.3150476217269897
1.1288199424743652
1.5215909481048584
1.4837346076965332
1.613781213760376
1.204599380493164
1.4013596773147583
1.3012316226959229
1.4193928241729736
1.4120984077453613
1.5305447578430176
1.1345930099487305
1.7234992980957031
1.3214647769927979
1.1774041652679443
1.3551340103149414
1.4999287128448486
1.5302281379699707
1.5595601797103882
1.219180941581726
1.6570810079574585
1.294635534286499
1.5315229892730713
1.3292533159255981
1.0792322158813477
1.1959378719329834
1.497417688369751
1.3582236766815186
1.4361295700073242
1.270830512046814
1.3805090188980103
1.2364100217819214
1.0180188417434692
1.3608064651489258
1.284928798675537
1.6965926885604858
1.9937623739242554
1.7821252346038818
1.1066519021987915
1.413153886795044
1.3679864406585693
1.193337321281433
1.0796024799346924
1.465

1.284699559211731
1.523740530014038
1.4939228296279907
1.5741691589355469
1.2369996309280396
1.171858310699463
1.5080225467681885
1.0874940156936646
0.8060487508773804
1.4859627485275269
1.3580213785171509
1.2437803745269775
1.3095223903656006
1.3997328281402588
1.3951563835144043
1.4599652290344238
1.4059466123580933
1.6317908763885498
1.6226683855056763
1.2446447610855103
1.3645681142807007
1.647036075592041
1.3972408771514893
1.2643587589263916
1.4086289405822754
1.3025267124176025
1.2144845724105835
1.4331449270248413
1.2063758373260498
1.3731626272201538
1.3350123167037964
1.2761379480361938
0.9937897324562073
1.3734195232391357
1.5110955238342285
1.4775418043136597
1.5824675559997559
1.4302724599838257
1.0771039724349976
1.4311957359313965
1.3233301639556885
1.6500253677368164
1.3853199481964111
1.4850175380706787
0.0
1.3810717541666773
621.0591430664062
621.0604248046875
621.0598754882812
621.0595092773438
621.0601196289062
621.0595703125
621.0599975585938
621.0596923828125
621.